In [1]:
import os
import numpy as np
import pandas as pd

# Словарь для хранения данных
data = {}

# Список состояний системы и соответствующие пути к данным
states = [
    ['Healthy_1_100', os.path.normpath(r'dataset/Electric/Motor-2/100/healthy 1'), None],
    ['Healthy_2_100', os.path.normpath(r'dataset/Electric/Motor-2/100/healthy 2'), None],
    ['Healthy_3_100', os.path.normpath(r'dataset/Electric/Motor-2/100/healthy 3'), None],
    ['BPFI', os.path.normpath(r'dataset/Electric/Motor-2/100/bearing bpfi 3'), None],
    ['BPFO',os.path.normpath(r'dataset/Electric/Motor-2/100/bearing bpfo 3'), None],
    ['BSF', os.path.normpath(r'dataset/Electric/Motor-2/100/bearing bsf'), None],
]

In [2]:
# Костыль на создание списков лейблов для присоединения к DataFrame'ам. Включает длину каждого сета из .csv файлов
labels = [
    np.zeros(10800000),
    np.zeros(9000000),
    np.zeros(8700000),
    np.ones(1500000),
    np.array([2 for k in range(2100000)]),
    np.array([3 for k in range(1500000)])
]
labels

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([1., 1., 1., ..., 1., 1., 1.]),
 array([2, 2, 2, ..., 2, 2, 2]),
 array([3, 3, 3, ..., 3, 3, 3])]

In [3]:
data_iter = []
i = 0
for state in states:
    print(state[0])  # Выводим текущее состояние системы
    path = state[1]  # Путь к данным

    # Проверяем, существует ли директория
    if not os.path.exists(path):
        print(f"Директория не существует: {path}")
        continue

    columns_name = []
    channels_data = []

    # Проходим по файлам в директории
    for ch_file in os.listdir(path):
        # Загружаем данные из файла, убираем колонку 'time'
        channel = pd.read_csv(os.path.join(path, ch_file)).drop(columns='time')
        ch_num = int(ch_file[-5:-4])  # Определяем номер канала
        columns_name.append(f'i_{ch_num}' if ch_num < 4 else f'v_{ch_num-3}')
        channels_data.append(channel.to_numpy().astype(np.float32).flatten(order='F'))

    ch_shape = channel.shape
    # Генерируем labels
    columns_name.append('label')
    channels_data.append(labels[i])
    # Формируем DataFrame и сохраняем его в словарь
    channels_data = pd.DataFrame(np.vstack(channels_data).T, columns=columns_name)
    data_iter.append(channels_data)
    print(channels_data)
    i += 1

Healthy_1_100
                 v_1        i_3         v_2        i_2         v_3        i_1  \
0        -125.253738 -24.287834  330.519867  22.242159 -192.438904   3.008112   
1        -172.212433 -24.350483  331.956482  22.851290 -190.253326   2.451916   
2        -163.295639 -24.382334  337.171875  23.583891 -184.354675   1.823028   
3        -168.528625 -24.521708  336.399567  24.197058 -184.450928   1.305648   
4        -181.449966 -24.620201  339.193451  24.997971 -180.946472   0.647593   
...              ...        ...         ...        ...         ...        ...   
10799995  187.444382 -23.758322  184.439835  -1.999077 -340.498993  26.017851   
10799996  180.922272 -24.504080  182.585968  -1.332109 -347.319000  26.106604   
10799997  175.655731 -25.214413  180.371811  -0.725094 -350.073486  26.237461   
10799998  115.721046 -25.602055  186.896454   0.013460 -344.033966  25.805588   
10799999  124.727783 -25.742361  201.877289   0.980156 -329.480011  25.043806   

          lab

In [4]:
# ПРОЦЕДУРА НЕОБЯЗАТЕЛЬНА и занимает некоторое время

# собираем все DataFrame'ы в один DataFrame 
data_full = pd.concat([data_iter[i] for i in range(len(data_iter))], ignore_index=True, sort=False)
data_full

,v_1,i_3,v_2,i_2,v_3,i_1,label
0,-125.253738,-24.287834,330.519867,22.242159,-192.438904,3.008112,0.0
1,-172.212433,-24.350483,331.956482,22.851290,-190.253326,2.451916,0.0
2,-163.295639,-24.382334,337.171875,23.583891,-184.354675,1.823028,0.0
3,-168.528625,-24.521708,336.399567,24.197058,-184.450928,1.305648,0.0
4,-181.449966,-24.620201,339.193451,24.997971,-180.946472,0.647593,0.0
...,...,...,...,...,...,...,...
33599995,130.061783,23.986174,-333.921936,-20.202623,204.804733,-4.085264,3.0
33599996,167.235229,23.966404,-345.413116,-21.205179,193.734528,-3.079870,3.0
33599997,121.537064,24.216984,-342.756134,-21.803228,196.691345,-2.778104,3.0
33599998,174.552399,24.394741,-344.987091,-22.651936,194.642563,-2.081206,3.0


In [5]:
def split_measurements(measurements):
    num_measurements = len(measurements)
    train_measurements = measurements[:int(0.75 * num_measurements)]
    test_measurements = measurements[int(0.75 * num_measurements):]
    return train_measurements, test_measurements

# Разделяем каждый DataFrame на два DataFrame'a: тренировочный и тестовый
train_full, test_full = [], []
for i in range(len(data_iter)):
    train_full.append(split_measurements(data_iter[i])[0])
    test_full.append(split_measurements(data_iter[i])[1])

In [6]:
# Разделяем и объединяем все DataFrame'ы в два: тренировочный и тестовый
train_data = pd.concat([train_full[i] for i in range(len(train_full))])
print('тренировочный сет готов')
test_data = pd.concat([test_full[i] for i in range(len(test_full))])
print('тестовый сет готов')

тренировочный сет готов
тестовый сет готов


In [7]:
train_data

,v_1,i_3,v_2,i_2,v_3,i_1,label
0,-125.253738,-24.287834,330.519867,22.242159,-192.438904,3.008112,0.0
1,-172.212433,-24.350483,331.956482,22.851290,-190.253326,2.451916,0.0
2,-163.295639,-24.382334,337.171875,23.583891,-184.354675,1.823028,0.0
3,-168.528625,-24.521708,336.399567,24.197058,-184.450928,1.305648,0.0
4,-181.449966,-24.620201,339.193451,24.997971,-180.946472,0.647593,0.0
...,...,...,...,...,...,...,...
1124995,-196.251724,19.258904,-169.826599,6.020471,330.256134,-24.808594,3.0
1124996,-192.240387,19.740215,-114.705887,5.598088,334.540070,-24.816151,3.0
1124997,-195.945770,20.141342,-147.620377,5.189390,331.059540,-24.906719,3.0
1124998,-186.288254,20.821888,-158.291397,4.508002,340.789490,-24.837933,3.0


In [8]:
test_data

,v_1,i_3,v_2,i_2,v_3,i_1,label
8100000,-175.380417,-25.504545,353.386688,26.587744,-162.427368,0.029366,0.0
8100001,-183.088791,-25.100723,346.136505,26.878139,-123.269989,-0.575861,0.0
8100002,-185.377029,-25.113119,343.601715,27.319870,-159.876831,-1.141877,0.0
8100003,-186.110352,-24.915501,342.752808,27.745775,-115.170792,-1.679566,0.0
8100004,-197.433716,-24.352638,331.854797,27.726664,-120.818130,-2.294428,0.0
...,...,...,...,...,...,...,...
1499995,130.061783,23.986174,-333.921936,-20.202623,204.804733,-4.085264,3.0
1499996,167.235229,23.966404,-345.413116,-21.205179,193.734528,-3.079870,3.0
1499997,121.537064,24.216984,-342.756134,-21.803228,196.691345,-2.778104,3.0
1499998,174.552399,24.394741,-344.987091,-22.651936,194.642563,-2.081206,3.0


In [9]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Импорт необходимых модулей из FEDOT
from fedot.core.repository.tasks import Task, TaskTypesEnum
from fedot.core.repository.dataset_types import DataTypesEnum
from fedot.core.data.data import InputData
from fedot.api.main import Fedot

In [10]:
# Разделяем данные на X и y
X_train = train_data.drop(columns=['label'])
y_train = train_data['label']
X_test = test_data.drop(columns=['label'])
y_test = test_data['label']

# Выводим информацию о сформированных наборах
print(f"Размер обучающей выборки: {X_train.shape}")
print(f"Размер тестовой выборки: {X_test.shape}")

Размер обучающей выборки: (25200000, 6)
Размер тестовой выборки: (8400000, 6)


In [ ]:
# Преобразуем DataFrame в numpy массив перед передачей в FEDOT
X_train_np = X_train.values
X_test_np = X_test.values

# Создание объекта InputData для обучающей выборки
train_input = InputData(idx=np.arange(0, len(X_train_np)),
                        features=X_train_np,
                        target=y_train.values,  # Преобразование y_train в numpy массив
                        task=Task(TaskTypesEnum.classification),
                        data_type=DataTypesEnum.table)

# Создание объекта InputData для тестовой выборки
test_input = InputData(idx=np.arange(0, len(X_test_np)),
                       features=X_test_np,
                       target=y_test.values,  # Преобразование y_test в numpy массив
                       task=Task(TaskTypesEnum.classification),
                       data_type=DataTypesEnum.table)

# Настройка FEDOT с пресетом на максимальную производительность без параметра cache
pipeline = Fedot(problem='classification', preset='auto', timeout=10)

# Обучение модели на обучающей выборке
pipeline.fit(train_input)

# Сохранение обученной модели
pipeline.current_pipeline.save(path="fedot_pipeline_anomaly_detection")


In [ ]:
import numpy as np
import pandas as pd
from fedot.api.main import Fedot
from fedot.core.data.data import InputData
from fedot.core.data.data import Task, TaskTypesEnum, DataTypesEnum
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns

# Предсказание на тестовой выборке
y_pred = pipeline.predict(test_input)

# Метрики на тестовой выборке
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
matrix = confusion_matrix(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print(f'Accuracy на тестовой выборке: {accuracy}')
print(f'F1-score на тестовой выборке: {f1}')
print(f'Precision на тестовой выборке: {precision}')
print(f'Recall на тестовой выборке: {recall}')
print('Classification Report:\n', report)

# Визуализация матрицы ошибок
plt.figure(figsize=(10, 7))
sns.heatmap(matrix, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix (Тестовая выборка)')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# Визуализация конфигурации пайплайна
pipeline.current_pipeline.show()

In [ ]:
# Сохранение обученной модели
pipeline.current_pipeline.save(path="fedot_pipeline_anomaly_detection (0-3 raw)")

# Сохранение предсказаний на тестовой выборке в CSV файл
output_path = "fedot_test_predictions (0-3 raw).csv"
pd.DataFrame({'True Labels': y_test.values, 'Predictions': y_pred.flatten()}).to_csv(output_path, index=False)
print(f"Результаты сохранены в {output_path}")